In [ ]:
# https://datascienceschool.net/view-notebook/8d34d65bcced42ef84996b5d56321ba9/

# Inception 모듈 정의
def inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32):
    """
    # Arguments
    x: 입력이미지
    o_1 : 1x1 conv 연산 출력값의 채널 수
    r_3 : 3x3 conv 이전에 있는 1x1 conv의 출력값 채널 수
    o_3 : 3x3 conv 연산 출력값의 채널 수
    r_5 : 5x5 conv 이전에 있는 1x1 conv의 출력값 채널 수
    o_5 : 5x5 conv 연산 출력값의 채널 수
    pool : maxpooling 다음의 1x1 conv의 출력값 채널 수

    # returns
    4 종류의 연산의 결과 값을 채널 방향으로 합친 결과
    """

    x_1 = layers.Conv2D(o_1, 1, padding='same')(x)
    
    x_2 = layers.Conv2D(r_3, 1, padding='same')(x)
    x_2 = layers.Conv2D(o_3, 3, padding='same')(x_2)
    
    x_3 = layers.Conv2D(r_5, 1, padding='same')(x)
    x_3 = layers.Conv2D(o_5, 1, padding='same')(x_3)
    
    x_4 = layers.MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(x)
    x_4 = layers.Conv2D(pool, 1, padding='same')(x_4)
    
    return layers.concatenate([x_1, x_2, x_3, x_4])

In [ ]:
from keras import layers
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img_shape = (224, 224, 3)

input_ = layers.Input(shape=img_shape)
x = layers.Conv2D(64, 7, strides=2, padding='same')(input_)
x = layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 1, strides=1)(x)
x = layers.Conv2D(192, 3, strides=1, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

x = inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32)
x = inception_module(x, o_1=128, r_3=128, o_3=192, r_5=32, o_5=96, pool=64)
x = layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

x = inception_module(x, o_1=192, r_3=96, o_3=208, r_5=16, o_5=48, pool=64)
x = inception_module(x, o_1=160, r_3=112, o_3=224, r_5=24, o_5=64, pool=64)
x = inception_module(x, o_1=128, r_3=128, o_3=256, r_5=24, o_5=64, pool=64)
x = inception_module(x, o_1=112, r_3=144, o_3=288, r_5=32, o_5=64, pool=64)
x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
x = layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
x = inception_module(x, o_1=384, r_3=192, o_3=384, r_5=48, o_5=128, pool=128)
x = layers.AveragePooling2D(pool_size=(7,7), strides=1)(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1000)(x)
output = layers.Activation('softmax')(x)

googlenet = Model(input_, output)

In [ ]:
googlenet.summary()

In [ ]:
print("전체 파라미터 수 : {}".format(sum([arr.flatten().shape[0] for arr in googlenet.get_weights()])))

# Inception V3

In [ ]:
from keras.applications.inception_v3 import InceptionV3, decode_predictions

inceptionv3 = InceptionV3(input_shape=(299,299,3))
inceptionv3.summary()

In [ ]:
import cv2
import time

img = cv2.imread('./sample_data/0.jpg', -1)
# plt.imshow(img)
# plt.show()
# print(img.shape)
img = cv2.resize(img, (299,299))

start = time.time()
yhat = inceptionv3.predict(img.reshape(-1, 299, 299, 3))
time = time.time() - start
label_key = np.argmax(yhat)
label = decode_predictions(yhat)
label = lable[0][0]

print("테스트 시 소요 시간 : {}".format(time))
print("%s (%.2f%%)" % (label[1], label[2]*100))
img = img[:,:,::-1]
plt.figure(figsize=(11,11))
plt.imshow(img)
plt.axis("off")
plt.show()

* 에러 메세지
UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_229/convolution}}]]
     
* 해결책
https://github.com/tensorflow/tensorflow/issues/28326
This error is for sure due to version mismatch of Cuda/cudnn. When I got this error I followed the instructions on this link https://www.tensorflow.org/install/gpu and it solved the problem. Have you tried that?

가상환경을 쓰자.....

In [1]:
import tensorflow as tf
tf.__version__

/home/test/opencv/InceptionV1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/test/opencv/InceptionV1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/test/opencv/InceptionV1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/test/opencv/InceptionV1/lib/python3.6/site-packages/ten

'1.13.1'